# Agent Totals Per Vendor

## Import Modules

In [ ]:
# Data Shaping
import pandas as pd
import numpy as np
import re
import datetime as dt
import math

# Visualization
[REDACTED]/.pyplot as plt
import seaborn as sns

# File Handling and Export
import json
import csv
import xlrd

In [ ]:
# add current timestamp to filename for reference
current_time = (dt.datetime.utcnow().strftime('%Y_%m_%d_%H%M%S'))

# git repo folder
git_folder = 'd:/git/example_infrastructure_data_dev'

# export folder will contain all csv exported DataFrames for Ticket Creation
export_folder = 'd:/exports/'

### Import Modules, Create env Variables

In [ ]:
from ts_api_connections import datto_rmm_api

In [ ]:

datto = datto_rmm_api.DattoRMM()
df_datto = datto.create_devices_dataframe()

In [ ]:
### Rename values to match Sophos : 'computer' or 'server'
df_datto['category'].replace({'Desktop':'computer','Laptop':'computer','Server':'server'},inplace=True)

## Add column 'system' with dattormm for compare
df_datto['system'] = 'dattormm'

## Data Shaping

In [ ]:
site_totals = pd.DataFrame(df_datto[['siteName', 'category']].groupby('siteName').value_counts())

In [ ]:
site_totals = site_totals.unstack(level=1)

In [ ]:
site_totals = site_totals.droplevel(level=0, axis=1)

In [ ]:
#site_totals.replace({np.NAN: 0})
site_totals.fillna(0, inplace=True)
site_counts = []

In [ ]:
for item in site_totals.iterrows():
    site = item[0]
    computers = math.ceil(item[1][0])
    servers = math.ceil(item[1][1])
    total = computers + servers
    site_counts.append({'siteName': site, 'computer': computers, 'servers': servers, 'total': total})
df_datto_counts = pd.DataFrame(site_counts)

## Datto Agent Totals

In [ ]:
df_datto.to_csv(f'{export_folder}[REDACTED]/.csv')

In [ ]:
df_datto_counts.to_csv(f'{export_folder}[REDACTED]/.csv')

# Sophos AV

# Sophos DataFrame Creation

In [ ]:
from ts_api_connections import sophos_central_api
sophos = sophos_central_api.SophosCentral()

### Create Sophos DataFrame from dictionary created via API iteration

In [ ]:
df_sophos_av = sophos.device_dataframe()

In [ ]:
df_sophos_av['site'].unique()

### Type | Category

In [ ]:
df_sophos_av['category'] = df_sophos_av['type']
df_sophos_av.drop(columns='type',inplace=True)


In [ ]:
df_sophos_av['category'].unique()

## Data Shaping

In [ ]:
site_totals = pd.DataFrame(df_sophos_av[['site', 'category']].groupby('site').value_counts())

In [ ]:
site_totals = site_totals.unstack(level=1)

In [ ]:
site_totals = site_totals.droplevel(level=0, axis=1)

In [ ]:
#site_totals.replace({np.NAN: 0})
site_totals.fillna(0, inplace=True)
site_counts = []

In [ ]:
for item in site_totals.iterrows():
    site = item[0]
    computers = math.ceil(item[1][0])
    servers = math.ceil(item[1][1])
    total = computers + servers
    site_counts.append({'siteName': site, 'computers': computers, 'servers': servers, 'total': total})
df_sophos_counts = pd.DataFrame(site_counts)

In [ ]:
df_sophos_counts